In [ ]:
from Raspi_MotorHAT import Raspi_MotorHAT
import keyboard
import threading
import time
import cv2
import RPi.GPIO as GPIO
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
mh = Raspi_MotorHAT(addr=0x6f)


def motor_go():   # 직진하는 함수
    motor1 = mh.getMotor(1)
    motor1.setSpeed(50)
    motor1.run(Raspi_MotorHAT.FORWARD)
    motor2 = mh.getMotor(2)
    motor2.setSpeed(50)
    motor2.run(Raspi_MotorHAT.FORWARD)
    motor3 = mh.getMotor(3)
    motor3.setSpeed(50)
    motor3.run(Raspi_MotorHAT.FORWARD)
    motor4 = mh.getMotor(4)
    motor4.setSpeed(50)
    motor4.run(Raspi_MotorHAT.FORWARD)

def motor_stop():  # 정지하는 함수
    motor1 = mh.getMotor(1)
    motor1.setSpeed(50)
    motor1.run(Raspi_MotorHAT.RELEASE)
    motor2 = mh.getMotor(2)
    motor2.setSpeed(50)
    motor2.run(Raspi_MotorHAT.RELEASE)
    motor3 = mh.getMotor(3)
    motor3.setSpeed(50)
    motor3.run(Raspi_MotorHAT.RELEASE)
    motor4 = mh.getMotor(4)
    motor4.setSpeed(50)
    motor4.run(Raspi_MotorHAT.RELEASE)
    
def motor_right():  # 우회전 함수
    motor1 = mh.getMotor(1)
    motor1.setSpeed(70)
    motor1.run(Raspi_MotorHAT.FORWARD)
    motor2 = mh.getMotor(2)
    motor2.setSpeed(15)
    motor2.run(Raspi_MotorHAT.FORWARD)
    motor3 = mh.getMotor(3)
    motor3.setSpeed(70)
    motor3.run(Raspi_MotorHAT.FORWARD)
    motor4 = mh.getMotor(4)
    motor4.setSpeed(15)
    motor4.run(Raspi_MotorHAT.FORWARD)  
    
def motor_left():   # 좌회전 함수
    motor1 = mh.getMotor(1)
    motor1.setSpeed(15)
    motor1.run(Raspi_MotorHAT.FORWARD)
    motor2 = mh.getMotor(2)
    motor2.setSpeed(70)
    motor2.run(Raspi_MotorHAT.FORWARD)
    motor3 = mh.getMotor(3)
    motor3.setSpeed(15)
    motor3.run(Raspi_MotorHAT.FORWARD)
    motor4 = mh.getMotor(4)
    motor4.setSpeed(70)
    motor4.run(Raspi_MotorHAT.FORWARD)
        


def img_preprocess(image):                         # 이미지를 학습된 데이터 형식에 맞게 보정하는 함수
    height, _, _ = image.shape
    image = image[int(height/2):,:,:]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)  # 트랙을 보다 선명하게
    image = cv2.resize(image, (200,66))
    image = cv2.GaussianBlur(image,(5,5),0)         # 트랙을 선명하게 필터 추가
    _,image = cv2.threshold(image,160,255,cv2.THRESH_BINARY_INV)  # 화질의 개선처리, 임계점에 따라 화질이 달라짐, 숫자를 조정하며 최적의 임계점을 찾는다.
    image = image / 255                             # 이미지 정규화
    return image

camera = cv2.VideoCapture(-1)
camera.set(3, 640)
camera.set(4, 480)

        
def main():
    
    model_path = '/home/jh/model/1205-2.h5'         # 학습 모델이 위치한 경로를 지정한다.
    model = load_model(model_path)                  # 모델을 불러온다.
    
    carState = "stop"
    
    try:
        while True:
            keyValue = cv2.waitKey(1)
        
            if keyboard.is_pressed("p"):
                break
            elif keyboard.is_pressed("w"):
                print("go")
                carState = "go"
            elif keyboard.is_pressed("s"):
                print("stop")
                carState = "stop"
                
            _, image = camera.read()
            preprocessed = img_preprocess(image)           # 보정한 이미지를 저장
            cv2.imshow('pre', preprocessed)                # 보정한 이미지를 opencv창으로 출력
            
            X = np.asarray([preprocessed])                 # X값에 형식에 맞춰 데이터를 넣는다.
            steering_angle = int(model.predict(X)[0])      # 각도를 예측해준다(조향각도)
            print("predict angle:",steering_angle)         # 예측한 각도를 출력
                
            if carState == "go":
                if steering_angle >= 80 and steering_angle <= 105: # 조향각도가 80이상 105이하라면 계속 직진
                    print("go")
                    motor_go()
                elif steering_angle > 105:                         # 조향각도가 105 이상일때 우회전
                    print("right")
                    motor_right()
                elif steering_angle < 80:                          # 조향각도가 80 이상일때 좌회전
                    print("left")
                    motor_left()
            elif carState == "stop":
                motor_stop()
            
    except KeyboardInterrupt:
        pass

if __name__ == '__main__':
    main()
    cv2.destroyAllWindows()
